### Extrair todas as tabelas do pdf. retorna uma lista de tabela.

In [2]:
import pdfplumber
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path  # Melhor forma para trabalhar com caminhos
import tabula
import fitz  # PyMuPDF
import re

In [ ]:
# Configurações
arquivo_pdf = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_02_DATASETS\demonstracao_finacneira\Demonstrações Financeiras 2024.pdf'

doc = fitz.open(arquivo_pdf)
paginas_selecionadas = [4]  # Páginas 1, 3 e 5 (índices começam em 0)

for num_pag in paginas_selecionadas:
    pagina = doc.load_page(num_pag)  # Carrega a página pelo número
    texto = pagina.get_text()
    print(f"\n--- Página {num_pag + 1} ---\n{texto}")


In [3]:

arquivo_pdf = r'E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_02_DATASETS\demonstracao_finacneira\Demonstrações Financeiras 2024.pdf'
doc = fitz.open(arquivo_pdf)
pagina = doc.load_page(4)

# Extrai os blocos de texto com coordenadas (x0, y0, x1, y1, texto, ...)
blocos = pagina.get_text("blocks")

# Ordena os blocos por posição vertical (y) e depois horizontal (x)
blocos_ordenados = sorted(blocos, key=lambda b: (round(b[1], 1), b[0]))

# Filtra apenas as linhas da tabela (heurística: só linhas com números e colunas conhecidas)
linhas_tabela = [b[4].strip() for b in blocos_ordenados if any(char.isdigit() for char in b[4])]

# Junta linhas consecutivas por altura semelhante (rústico mas eficaz)
tabela_agrupada = []
linha_atual = ''
ultima_y = None

for bloco in blocos_ordenados:
    texto = bloco[4].strip()
    if not texto:
        continue
    y = round(bloco[1], 1)
    if ultima_y is None or abs(y - ultima_y) < 3:
        linha_atual += ' ' + texto
    else:
        tabela_agrupada.append(linha_atual.strip())
        linha_atual = texto
    ultima_y = y
tabela_agrupada.append(linha_atual.strip())

# Filtra só as linhas que parecem fazer parte da tabela
tabela_final = [linha for linha in tabela_agrupada if any(c.isdigit() for c in linha)]

# Visualiza as linhas extraídas
for linha in tabela_final:
    print(linha)


WEG S.A.     CNPJ 84.429.695/0001-11   |   CVM 00541-0 Companhia Aberta 
 
 
RELATÓRIO DA ADMINISTRAÇÃO
Em 31 de dezembro de 2024
O Custo dos Produtos Vendidos (CPV) apresentou crescimento de 16,0%, atingindo R$ 25.173,1 milhões, 
representando uma margem bruta de 33,7%. O mix de produtos vendidos mais favorável, em conjunto com a constante 
busca por eficiência operacional que proporcionaram ganhos de produtividade, contribuíram de maneira relevante para 
o crescimento das margens operacionais da Companhia durante o ano, apesar da variação dos custos de algumas 
matérias-primas que compõem nossa estrutura de custos.
As Despesas com Vendas, Gerais e Administrativas consolidadas totalizaram R$ 4.286,7 milhões, um aumento de 
23,5% em relação a 2023. Quando analisadas em função da receita operacional as despesas representam 11,3%, um 
aumento de 0,6 ponto percentual em relação ao ano anterior. A Companhia segue com ações para aumentar a 
produtividade, incluindo melhorias nos processos e

In [6]:
import fitz  # PyMuPDF
import pandas as pd

# Caminho do arquivo PDF
arquivo_pdf = r"E:\PYTHON\PROJETOS_PESSOAIS\analise_weg\_02_DATASETS\demonstracao_finacneira\Demonstrações Financeiras 2024.pdf"

# Abre o PDF e carrega a página onde está a tabela (ex: página 5 = índice 4)
doc = fitz.open(arquivo_pdf)
pagina = doc.load_page(4)

# Extrai as linhas do texto da página
linhas = pagina.get_text().splitlines()

# Filtra apenas o conteúdo abaixo da linha "Tabela 1"
inicio_tabela = False
linhas_filtradas = []

for linha in linhas:
    if "Tabela 1" in linha:
        inicio_tabela = True
        continue
    if inicio_tabela:
        if "RESULTADO FINANCEIRO" in linha:
            break  # fim da tabela
        if linha.strip():  # ignora linhas em branco
            linhas_filtradas.append(linha.strip())

# Agrupa os dados da tabela (1 linha de título + 5 linhas de valores)
dados = []
i = 0
while i < len(linhas_filtradas):
    nome = linhas_filtradas[i]
    valores = linhas_filtradas[i+1:i+6]
    if len(valores) == 5:
        dados.append([nome] + valores)
    else:
        print(f"⚠️ Erro ao processar linha: {nome} → {valores}")
    i += 6

# Cria DataFrame
colunas = ["Indicador", "2024", "2023", "% Var 23/24", "2022", "% Var 22/23"]
df = pd.DataFrame(dados, columns=colunas)

# Conversão dos valores numéricos
def to_float(valor):
    if isinstance(valor, str):
        valor = valor.replace(".", "").replace(",", ".").replace("(", "-").replace(")", "")
        try:
            return float(valor)
        except:
            return valor
    return valor

for col in ["2024", "2023", "2022"]:
    df[col] = df[col].apply(to_float)

print(df)


                                           Indicador        2024        2023  \
0                                  Valores em R$ mil      2024.0      2023.0   
1                        Receita Operacional Líquida  37986941.0  32503601.0   
2                         Lucro Líquido do Exercício   6042593.0   5731670.0   
3  Lucro Líquido antes dos acionistas não control...   6318763.0   5867615.0   
4                                    (+) IRPJ e CSLL   1589745.0    723182.0   
5                         (+/-) Resultado Financeiro   -217980.0   -128672.0   
6                        (+) Depreciação/Amortização    812485.0    628042.0   
7                                             EBITDA   8503013.0   7090167.0   
8                                      Margem EBITDA       22.4%       21.8%   

  % Var 23/24        2022 % Var 22/23  
0           %      2022.0           %  
1       16,9%  29904722.0       27,0%  
2        5,4%   4208084.0       43,6%  
3        7,7%   4272872.0       47,9%  